In [141]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import heapq
import seaborn as sns
import copy
from collections import deque, defaultdict
from typing import List, Tuple, Dict, Set, Any, Callable, Optional
import json
import os
from datetime import datetime

# Cấu hình hiển thị cho matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.grid'] = True

# Cấu hình hiển thị
import warnings
warnings.filterwarnings('ignore')

In [142]:
# Lớp Puzzle
class Puzzle:
    def __init__(self, state, goal, size=3):
        self.state = tuple(state)
        self.goal = tuple(goal)
        self.size = size
        self.blank_position = self.state.index(0)
    
    def __eq__(self, other):
        return self.state == other.state
    
    def __hash__(self):
        return hash(self.state)
    
    def get_possible_moves(self):
        moves = []
        # Tính vị trí hàng, cột của ô trống
        blank_row = self.blank_position // self.size
        blank_col = self.blank_position % self.size
        
        # Tối ưu: Sử dụng list comprehension thay vì vòng lặp
        # Các hướng có thể di chuyển: lên, xuống, trái, phải
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        for dr, dc in directions:
            new_row, new_col = blank_row + dr, blank_col + dc
            
            # Kiểm tra xem vị trí mới có hợp lệ không
            if 0 <= new_row < self.size and 0 <= new_col < self.size:
                # Tính vị trí mới trong mảng 1D
                new_blank_pos = new_row * self.size + new_col
                
                # Tạo trạng thái mới bằng cách hoán đổi ô trống với ô bên cạnh
                new_state = list(self.state)
                new_state[self.blank_position], new_state[new_blank_pos] = new_state[new_blank_pos], new_state[self.blank_position]
                
                # Tối ưu: Sử dụng tuple trực tiếp cho trạng thái mới
                moves.append(Puzzle(tuple(new_state), self.goal, self.size))
        
        return moves
    
    def is_goal(self):
        return self.state == self.goal

In [143]:
# Lớp Node để lưu trữ thông tin trong quá trình tìm kiếm
class Node:
    __slots__ = ('puzzle', 'parent', 'action', 'path_cost', 'depth')  # Tối ưu bộ nhớ
    
    def __init__(self, puzzle, parent=None, action=None, path_cost=0, depth=0):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = depth
    
    def __lt__(self, other):
        return self.path_cost < other.path_cost
    
    def expand(self):
        # Tối ưu: Trực tiếp tạo và trả về list gồm các nodes con
        return [Node(
                puzzle=next_puzzle,
                parent=self,
                action=None,
                path_cost=self.path_cost + 1,
                depth=self.depth + 1
            ) for next_puzzle in self.puzzle.get_possible_moves()]
    
    def solution(self):
        path = []
        node = self
        while node:
            path.append(node.puzzle.state)
            node = node.parent
        return list(reversed(path))

In [144]:
# Hàm tạo 8-puzzle ngẫu nhiên có thể giải được
def generate_random_8puzzle():
    """
    Tạo một 8-puzzle ngẫu nhiên có thể giải được
    """
    # Trạng thái đích: 0 1 2 3 4 5 6 7 8
    goal_state = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
    
    # Tạo trạng thái ban đầu bằng cách xáo trộn ngẫu nhiên
    while True:
        initial_state = np.copy(goal_state)
        np.random.shuffle(initial_state)
        
        # Kiểm tra xem puzzle có thể giải được không
        if is_solvable(initial_state, goal_state):
            return initial_state.tolist()

# Hàm tạo 15-puzzle ngẫu nhiên có thể giải được
def generate_random_15puzzle():
    """
    Tạo một 15-puzzle ngẫu nhiên có thể giải được
    """
    # Trạng thái đích: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    goal_state = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
    
    # Tạo trạng thái ban đầu bằng cách xáo trộn ngẫu nhiên
    while True:
        initial_state = np.copy(goal_state)
        np.random.shuffle(initial_state)
        
        # Kiểm tra xem puzzle có thể giải được không
        if is_solvable_15puzzle(initial_state):
            return initial_state.tolist()

# Kiểm tra tính giải được của 8-puzzle
def is_solvable(puzzle, goal):
    # Tính số nghịch thế (inversion) của puzzle và goal
    inversions_puzzle = count_inversions(puzzle)
    inversions_goal = count_inversions(goal)
    
    # Puzzle có thể giải được khi và chỉ khi parity của số nghịch thế giống nhau
    return inversions_puzzle % 2 == inversions_goal % 2

# Đếm số nghịch thế - được tối ưu
def count_inversions(state):
    state_list = state.tolist() if isinstance(state, np.ndarray) else state
    # Bỏ qua ô trống (0) khi đếm nghịch thế
    state_without_blank = [x for x in state_list if x != 0]
    
    # Tối ưu: Sử dụng list comprehension để đếm nghịch thế
    inversions = sum(1 for i in range(len(state_without_blank)) 
                    for j in range(i+1, len(state_without_blank)) 
                    if state_without_blank[i] > state_without_blank[j])
    
    return inversions

# Kiểm tra tính giải được của 15-puzzle
def is_solvable_15puzzle(puzzle):
    """
    Kiểm tra xem một 15-puzzle có thể giải được không
    """
    # Tìm vị trí của ô trống (0)
    blank_row = np.where(puzzle.reshape(4, 4) == 0)[0][0]
    
    # Đếm số nghịch thế (inversion)
    inversions = 0
    for i in range(len(puzzle)):
        if puzzle[i] == 0:
            continue
        for j in range(i+1, len(puzzle)):
            if puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inversions += 1
    
    # Đối với 15-puzzle (4x4), puzzle có thể giải được khi:
    # - Nếu ô trống nằm trên hàng lẻ (tính từ dưới lên, 1-indexed), số nghịch thế phải là số chẵn
    # - Nếu ô trống nằm trên hàng chẵn (tính từ dưới lên, 1-indexed), số nghịch thế phải là số lẻ
    if (4 - blank_row) % 2 == 1:  # Hàng lẻ (tính từ dưới lên)
        return inversions % 2 == 0
    else:  # Hàng chẵn (tính từ dưới lên)
        return inversions % 2 == 1

In [145]:
# Tạo tập dữ liệu 8-puzzle
def create_8puzzle_datasets():
    """
    Tạo các tập dữ liệu 8-puzzle với các kích thước khác nhau
    """
    # Số lượng puzzle cho mỗi tập dữ liệu
    dataset_sizes = [10, 100, 1000, 10000]
    
    # Tạo thư mục lưu dữ liệu nếu chưa tồn tại
    if not os.path.exists('data8puzzle'):
        os.makedirs('data8puzzle')
    
    # Trạng thái đích cố định: 0 1 2 3 4 5 6 7 8
    goal_state = list(range(9))  # [0, 1, 2, 3, 4, 5, 6, 7, 8]
    
    # Tạo và lưu các tập dữ liệu
    for size in dataset_sizes:
        # Tạo dataset
        dataset = []
        for i in range(size):
            initial_state = generate_random_8puzzle()
            puzzle_data = {
                "id": i,
                "initial_state": initial_state,
                "goal_state": goal_state,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            dataset.append(puzzle_data)
        
        # Lưu dataset
        filename = f"data8puzzle/dataset_{size}.json"
        with open(filename, 'w') as f:
            json.dump(dataset, f, indent=4)
        
        print(f"Đã tạo và lưu {size} 8-puzzles vào file '{filename}'")

# Tạo tập dữ liệu 15-puzzle
def create_15puzzle_datasets():
    """
    Tạo các tập dữ liệu 15-puzzle với các kích thước khác nhau
    """
    # Số lượng puzzle cho mỗi tập dữ liệu
    dataset_sizes = [10, 100, 1000, 10000]
    
    # Tạo thư mục lưu dữ liệu nếu chưa tồn tại
    if not os.path.exists('data15puzzle'):
        os.makedirs('data15puzzle')
    
    # Trạng thái đích cố định: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    goal_state = list(range(16))  # [0, 1, 2, ..., 15]
    
    # Tạo và lưu các tập dữ liệu
    for size in dataset_sizes:
        # Tạo dataset
        dataset = []
        for i in range(size):
            initial_state = generate_random_15puzzle()
            puzzle_data = {
                "id": i,
                "initial_state": initial_state,
                "goal_state": goal_state,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            dataset.append(puzzle_data)
        
        # Lưu dataset
        filename = f"data15puzzle/dataset_{size}.json"
        with open(filename, 'w') as f:
            json.dump(dataset, f, indent=4)
        
        print(f"Đã tạo và lưu {size} 15-puzzles vào file '{filename}'")
# Tạo thư mục dữ liệu và sinh dữ liệu
if __name__ == "__main__":
    # Tạo dữ liệu 8-puzzle
    print("Đang tạo dữ liệu 8-puzzle...")
    create_8puzzle_datasets()
    
    # Tạo dữ liệu 15-puzzle
    print("Đang tạo dữ liệu 15-puzzle...")
    create_15puzzle_datasets()
    
    print("Hoàn thành tạo dữ liệu!")


Đang tạo dữ liệu 8-puzzle...
Đã tạo và lưu 10 8-puzzles vào file 'data8puzzle/dataset_10.json'
Đã tạo và lưu 100 8-puzzles vào file 'data8puzzle/dataset_100.json'
Đã tạo và lưu 1000 8-puzzles vào file 'data8puzzle/dataset_1000.json'
Đã tạo và lưu 10000 8-puzzles vào file 'data8puzzle/dataset_10000.json'
Đang tạo dữ liệu 15-puzzle...
Đã tạo và lưu 10 15-puzzles vào file 'data15puzzle/dataset_10.json'
Đã tạo và lưu 100 15-puzzles vào file 'data15puzzle/dataset_100.json'
Đã tạo và lưu 1000 15-puzzles vào file 'data15puzzle/dataset_1000.json'
Đã tạo và lưu 10000 15-puzzles vào file 'data15puzzle/dataset_10000.json'
Hoàn thành tạo dữ liệu!


In [146]:
# Đọc dữ liệu từ file JSON
def load_puzzle_data(file_path):
    """
    Đọc dữ liệu puzzle từ file JSON
    
    Args:
        file_path (str): Đường dẫn tới file JSON
        
    Returns:
        list: Danh sách các puzzle
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# ===== THUẬT TOÁN TÌM KIẾM KHÔNG THÔNG TIN =====

In [147]:
# 1. Breadth-First Search (BFS) - Tối ưu
def breadth_first_search(initial_puzzle, time_limit=60):
    # Khởi tạo nút gốc
    initial_node = Node(initial_puzzle)
    
    # Nếu trạng thái ban đầu đã là trạng thái đích
    if initial_puzzle.is_goal():
        return initial_node.solution(), {"nodes_explored": 1, "max_memory": 1, "solution_length": 1}
    
    start_time = time.time()
    
    # Khởi tạo hàng đợi
    frontier = deque([initial_node])
    # Tập hợp các trạng thái đã thăm - sử dụng set thay vì dict để tăng tốc
    explored = set()
    # Lưu trữ trạng thái của các nút trong frontier
    frontier_set = {initial_puzzle.state}
    
    # Thống kê
    nodes_explored = 0
    max_memory = 1  # Khởi tạo bằng 1 vì đã có nút gốc trong frontier
    
    while frontier:
        # Kiểm tra timeout
        if time.time() - start_time > time_limit:
            return None, {"nodes_explored": nodes_explored, "max_memory": max_memory, "solution_length": 0, "timeout": True}
            
        # Lấy nút đầu tiên từ hàng đợi
        node = frontier.popleft()
        frontier_set.remove(node.puzzle.state)
        
        # Thêm trạng thái vào tập hợp đã thăm
        explored.add(node.puzzle.state)
        nodes_explored += 1
        
        # Tối ưu: Chỉ mở rộng các nút cần thiết
        for child in node.expand():
            child_state = child.puzzle.state
            
            # Nếu trạng thái chưa được thăm và không có trong frontier
            if child_state not in explored and child_state not in frontier_set:
                # Kiểm tra xem có phải là trạng thái đích không
                if child.puzzle.is_goal():
                    solution_path = child.solution()
                    stats = {
                        "nodes_explored": nodes_explored,
                        "max_memory": max(max_memory, len(frontier) + 1),
                        "solution_length": len(solution_path),
                        "timeout": False
                    }
                    return solution_path, stats
                
                # Thêm nút con vào cuối hàng đợi
                frontier.append(child)
                frontier_set.add(child_state)
                max_memory = max(max_memory, len(frontier))
    
    # Không tìm thấy lời giải
    stats = {
        "nodes_explored": nodes_explored,
        "max_memory": max_memory,
        "solution_length": 0,
        "timeout": False
    }
    return None, stats

In [148]:
# 2. Depth-First Search (DFS) - Tối ưu
def depth_first_search(initial_puzzle, max_depth=100, time_limit=60):
    initial_node = Node(initial_puzzle)
    
    if initial_puzzle.is_goal():
        return initial_node.solution(), {"nodes_explored": 1, "max_memory": 1, "solution_length": 1, "timeout": False}
    
    start_time = time.time()
    
    # Sử dụng list thay vì deque để tăng tốc độ - cho DFS, list hiệu quả hơn
    frontier = [initial_node]
    explored = set()
    frontier_set = {initial_puzzle.state}
    
    nodes_explored = 0
    max_memory = 1
    
    while frontier:
        # Kiểm tra timeout
        if time.time() - start_time > time_limit:
            return None, {"nodes_explored": nodes_explored, "max_memory": max_memory, "solution_length": 0, "timeout": True}
            
        node = frontier.pop()
        frontier_set.remove(node.puzzle.state)
        
        explored.add(node.puzzle.state)
        nodes_explored += 1
        
        if node.depth >= max_depth:
            continue
        
        # Tối ưu: Chỉ tạo và xử lý các nút con có triển vọng
        for child in reversed(node.expand()):
            child_state = child.puzzle.state
            
            if child_state not in explored and child_state not in frontier_set:
                if child.puzzle.is_goal():
                    solution_path = child.solution()
                    stats = {
                        "nodes_explored": nodes_explored,
                        "max_memory": max(max_memory, len(frontier) + 1),
                        "solution_length": len(solution_path),
                        "timeout": False
                    }
                    return solution_path, stats
                
                frontier.append(child)
                frontier_set.add(child_state)
                max_memory = max(max_memory, len(frontier))
    
    stats = {
        "nodes_explored": nodes_explored,
        "max_memory": max_memory,
        "solution_length": 0,
        "timeout": False
    }
    return None, stats

In [149]:
# 3. Uniform-Cost Search - Tối ưu
def uniform_cost_search(initial_puzzle, time_limit=60):
    initial_node = Node(initial_puzzle)
    
    if initial_puzzle.is_goal():
        return initial_node.solution(), {"nodes_explored": 1, "max_memory": 1, "solution_length": 1, "timeout": False}
    
    start_time = time.time()
    
    # Sử dụng heapq với entry finder pattern để tăng hiệu suất
    frontier = []
    entry_finder = {}  # state -> (priority, count, node)
    counter = 0
    
    def add_to_frontier(node, priority):
        nonlocal counter
        if node.puzzle.state in entry_finder:
            old_priority = entry_finder[node.puzzle.state][0]
            if priority < old_priority:
                # Đánh dấu entry cũ là REMOVED (None)
                entry_finder[node.puzzle.state] = None
            else:
                return  # Giữ nguyên entry cũ
        entry = [priority, counter, node]
        entry_finder[node.puzzle.state] = entry
        heapq.heappush(frontier, entry)
        counter += 1
    
    # Thêm nút đầu tiên
    add_to_frontier(initial_node, 0)
    
    explored = set()
    nodes_explored = 0
    max_memory = 1
    
    while frontier:
        # Kiểm tra timeout
        if time.time() - start_time > time_limit:
            return None, {"nodes_explored": nodes_explored, "max_memory": max_memory, "solution_length": 0, "timeout": True}
            
        # Lấy nút có chi phí thấp nhất
        priority, _, node = heapq.heappop(frontier)
        
        # Bỏ qua nếu node này đã bị đánh dấu REMOVED
        if node.puzzle.state in entry_finder and entry_finder[node.puzzle.state] is None:
            continue
        
        # Bỏ qua nếu trạng thái đã được thăm
        if node.puzzle.state in explored:
            continue
        
        # Xóa khỏi entry_finder
        entry_finder.pop(node.puzzle.state, None)
        
        # Thêm vào explored
        explored.add(node.puzzle.state)
        nodes_explored += 1
        
        # Kiểm tra goal
        if node.puzzle.is_goal():
            solution_path = node.solution()
            stats = {
                "nodes_explored": nodes_explored,
                "max_memory": max(max_memory, len(frontier) + 1),
                "solution_length": len(solution_path),
                "timeout": False
            }
            return solution_path, stats
        
        # Mở rộng nút hiện tại
        for child in node.expand():
            if child.puzzle.state not in explored:
                add_to_frontier(child, child.path_cost)
                max_memory = max(max_memory, len(frontier))
    
    stats = {
        "nodes_explored": nodes_explored,
        "max_memory": max_memory,
        "solution_length": 0,
        "timeout": False
    }
    return None, stats

In [150]:
def depth_limited_search(initial_puzzle, limit=30, time_limit=60):
    initial_node = Node(initial_puzzle)
    
    if initial_puzzle.is_goal():
        return initial_node.solution(), {"nodes_explored": 1, "max_memory": 1, "solution_length": 1, "timeout": False}
    
    start_time = time.time()
    
    frontier = [initial_node]
    frontier_set = {initial_puzzle.state}
    
    nodes_explored = 0
    max_memory = 1
    cutoff_occurred = False
    
    # Thay thế dict bằng defaultdict để tăng tốc
    from collections import defaultdict
    explored = defaultdict(lambda: float('inf'))
    explored[initial_puzzle.state] = 0
    
    while frontier:
        # Kiểm tra timeout
        if time.time() - start_time > time_limit:
            return None, {"nodes_explored": nodes_explored, "max_memory": max_memory, "solution_length": 0, "timeout": True}
            
        node = frontier.pop()
        frontier_set.remove(node.puzzle.state)
        
        nodes_explored += 1
        
        if node.depth < limit:
            for child in reversed(node.expand()):
                child_state = child.puzzle.state
                
                if child.depth < explored[child_state]:
                    explored[child_state] = child.depth
                    
                    if child.puzzle.is_goal():
                        solution_path = child.solution()
                        stats = {
                            "nodes_explored": nodes_explored,
                            "max_memory": max(max_memory, len(frontier) + 1),
                            "solution_length": len(solution_path),
                            "timeout": False
                        }
                        return solution_path, stats
                    
                    frontier.append(child)
                    frontier_set.add(child_state)
                    max_memory = max(max_memory, len(frontier))
        else:
            cutoff_occurred = True
    
    stats = {
        "nodes_explored": nodes_explored,
        "max_memory": max_memory,
        "solution_length": 0,
        "timeout": False
    }
    
    if cutoff_occurred:
        return "cutoff", stats
    else:
        return None, stats

In [151]:
def iterative_deepening_search(initial_puzzle, max_depth=30, time_limit=60):
    start_time = time.time()
    total_nodes_explored = 0
    max_memory_overall = 0
    
    for depth in range(max_depth + 1):
        # Kiểm tra timeout
        if time.time() - start_time > time_limit:
            stats_overall = {
                "nodes_explored": total_nodes_explored,
                "max_memory": max_memory_overall,
                "solution_length": 0,
                "final_depth": depth - 1,
                "timeout": True
            }
            return None, stats_overall
            
        # Truyền vào time limit còn lại
        remaining_time = time_limit - (time.time() - start_time)
        if remaining_time <= 0:
            stats_overall = {
                "nodes_explored": total_nodes_explored,
                "max_memory": max_memory_overall,
                "solution_length": 0,
                "final_depth": depth - 1,
                "timeout": True
            }
            return None, stats_overall
            
        result, stats = depth_limited_search(initial_puzzle, depth, remaining_time)
        
        total_nodes_explored += stats["nodes_explored"]
        max_memory_overall = max(max_memory_overall, stats["max_memory"])
        
        if result != "cutoff" or stats.get("timeout", False):
            stats_overall = {
                "nodes_explored": total_nodes_explored,
                "max_memory": max_memory_overall,
                "solution_length": stats["solution_length"] if result and result != "cutoff" else 0,
                "final_depth": depth,
                "timeout": stats.get("timeout", False)
            }
            return result if result != "cutoff" else None, stats_overall
    
    stats_overall = {
        "nodes_explored": total_nodes_explored,
        "max_memory": max_memory_overall,
        "solution_length": 0,
        "final_depth": max_depth,
        "timeout": False
    }
    return None, stats_overall

In [152]:
# 6. Bidirectional Search - Tối ưu
def bidirectional_search(initial_puzzle, time_limit=60):
    initial_node = Node(initial_puzzle)
    goal_puzzle = Puzzle(list(initial_puzzle.goal), list(initial_puzzle.goal), initial_puzzle.size)
    goal_node = Node(goal_puzzle)
    
    if initial_puzzle.is_goal():
        return initial_node.solution(), {
            "nodes_explored": 1, 
            "max_memory": 1, 
            "solution_length": 1,
            "timeout": False
        }
    
    start_time = time.time()
    
    # Sử dụng defaultdict để tăng tốc
    frontier_forward = deque([initial_node])
    explored_forward = {initial_puzzle.state: initial_node}
    
    frontier_backward = deque([goal_node])
    explored_backward = {goal_puzzle.state: goal_node}
    
    nodes_explored = 0
    max_memory = 2
    
    while frontier_forward and frontier_backward:
        # Kiểm tra timeout
        if time.time() - start_time > time_limit:
            return None, {
                "nodes_explored": nodes_explored,
                "max_memory": max_memory,
                "solution_length": 0,
                "forward_explored": len(explored_forward),
                "backward_explored": len(explored_backward),
                "timeout": True
            }
            
        # Ưu tiên mở rộng từ phía có ít nút đã thăm hơn
        if len(explored_forward) <= len(explored_backward):
            # Tìm kiếm từ phía trước
            node_forward = frontier_forward.popleft()
            nodes_explored += 1
            
            # Mở rộng nút từ phía trước
            for child in node_forward.expand():
                child_state = child.puzzle.state
                
                # Nếu trạng thái đã được thăm từ phía sau
                if child_state in explored_backward:
                    # Tìm thấy đường đi
                    backward_node = explored_backward[child_state]
                    
                    # Xây dựng đường đi
                    forward_path = list(child.solution()[:-1])
                    backward_path = list(reversed(backward_node.solution()[1:]))
                    
                    solution_path = forward_path + backward_path
                    
                    stats = {
                        "nodes_explored": nodes_explored,
                        "max_memory": max_memory,
                        "solution_length": len(solution_path),
                        "forward_explored": len(explored_forward),
                        "backward_explored": len(explored_backward),
                        "timeout": False
                    }
                    return solution_path, stats
                
                # Nếu trạng thái chưa được thăm từ phía trước
                if child_state not in explored_forward:
                    frontier_forward.append(child)
                    explored_forward[child_state] = child
                    max_memory = max(max_memory, len(frontier_forward) + len(frontier_backward))
        else:
            # Tìm kiếm từ phía sau
            node_backward = frontier_backward.popleft()
            nodes_explored += 1
                
            # Mở rộng nút từ phía sau
            for child in node_backward.expand():
                child_state = child.puzzle.state
                
                # Nếu trạng thái đã được thăm từ phía trước
                if child_state in explored_forward:
                    # Tìm thấy đường đi
                    forward_node = explored_forward[child_state]
                    
                    # Xây dựng đường đi
                    forward_path = list(forward_node.solution())
                    backward_path = list(reversed(child.solution()[1:]))
                    
                    solution_path = forward_path + backward_path
                    
                    stats = {
                        "nodes_explored": nodes_explored,
                        "max_memory": max_memory,
                        "solution_length": len(solution_path),
                        "forward_explored": len(explored_forward),
                        "backward_explored": len(explored_backward),
                        "timeout": False
                    }
                    return solution_path, stats
                
                # Nếu trạng thái chưa được thăm từ phía sau
                if child_state not in explored_backward:
                    frontier_backward.append(child)
                    explored_backward[child_state] = child
                    max_memory = max(max_memory, len(frontier_forward) + len(frontier_backward))
    
    # Không tìm thấy lời giải
    stats = {
        "nodes_explored": nodes_explored,
        "max_memory": max_memory,
        "solution_length": 0,
        "forward_explored": len(explored_forward),
        "backward_explored": len(explored_backward),
        "timeout": False
    }
    return None, stats

# ===== ĐÁNH GIÁ VÀ THỰC NGHIỆM =====

In [153]:
def visualize_puzzle_state(state, size):
    """
    Trực quan hóa trạng thái của puzzle với hiển thị nâng cao
    
    Args:
        state (list): Trạng thái của puzzle
        size (int): Kích thước của puzzle (3 cho 8-puzzle, 4 cho 15-puzzle)
    """
    # Tạo lưới
    grid = np.array(state).reshape(size, size)
    
    # Tạo hình
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Vẽ lưới
    ax.set_xlim(0, size)
    ax.set_ylim(0, size)
    ax.set_xticks(np.arange(0, size + 1, 1))
    ax.set_yticks(np.arange(0, size + 1, 1))
    ax.grid(True, color='gray', linestyle='-', linewidth=1.5)
    
    # Đổi ngược trục y
    ax.invert_yaxis()
    
    # Tạo colormap
    cmap = plt.cm.viridis
    
    # Vẽ các ô
    for i in range(size):
        for j in range(size):
            val = grid[i, j]
            if val != 0:  # Bỏ qua ô trống
                # Tính màu dựa trên giá trị
                color = cmap(val / (size**2 - 1))
                rect = plt.Rectangle((j, i), 1, 1, facecolor=color, alpha=0.7, edgecolor='black', linewidth=1.5)
                ax.add_patch(rect)
                
                # Vẽ số trong các ô
                ax.text(j + 0.5, i + 0.5, str(val), va='center', ha='center', 
                       fontsize=20, fontweight='bold', color='white')
    
    # Tô màu ô trống
    blank_row, blank_col = np.where(grid == 0)
    if len(blank_row) > 0 and len(blank_col) > 0:
        blank_row, blank_col = blank_row[0], blank_col[0]
        rect = plt.Rectangle((blank_col, blank_row), 1, 1, facecolor='lightgray', 
                           alpha=0.5, edgecolor='black', linewidth=2, linestyle='--')
        ax.add_patch(rect)
    
    # Thêm tiêu đề
    ax.set_title("Trạng thái puzzle", fontsize=16, fontweight='bold')
    
    # Loại bỏ nhãn trục
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    # Thêm thông tin
    goal_order = list(range(size * size))
    if tuple(state) != tuple(goal_order):
        misplaced = sum(1 for i in range(len(state)) if state[i] != 0 and state[i] != i)
        manhattan = sum(abs(i // size - state[i] // size) + abs(i % size - state[i] % size) 
                      for i in range(len(state)) if state[i] != 0)
        
        info_text = f"Số ô sai vị trí: {misplaced}\nKhoảng cách Manhattan: {manhattan}"
        plt.figtext(0.1, 0.02, info_text, fontsize=12)
    
    ax.set_aspect('equal')
    plt.tight_layout()
    
    return fig

In [154]:
def visualize_solution_path(path, size):
    """
    Trực quan hóa đường đi từ trạng thái ban đầu đến trạng thái đích với hiển thị nâng cao
    
    Args:
        path (list): Đường đi (danh sách các trạng thái)
        size (int): Kích thước của puzzle (3 cho 8-puzzle, 4 cho 15-puzzle)
    """
    # Số lượng trạng thái
    n_states = len(path)
    
    # Tính số hàng và cột để hiển thị tất cả các bước
    cols = min(4, n_states)  # Tối đa 4 cột
    rows = (n_states - 1) // cols + 1
    
    # Tạo hình lớn với kích thước phù hợp
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    
    # Làm phẳng mảng axes nếu cần
    if rows > 1 or cols > 1:
        axes = axes.flatten()
    else:
        axes = [axes]  # Đảm bảo axes luôn là một danh sách
    
    # Tạo colormap
    cmap = plt.cm.viridis
    
    # Vẽ từng trạng thái
    for i, state in enumerate(path):
        if i < len(axes):
            ax = axes[i]
            
            # Tạo lưới
            grid = np.array(state).reshape(size, size)
            
            # Vẽ lưới
            ax.set_xlim(0, size)
            ax.set_ylim(0, size)
            ax.set_xticks(np.arange(0, size + 1, 1))
            ax.set_yticks(np.arange(0, size + 1, 1))
            ax.grid(True, color='gray', linestyle='-', linewidth=1.5)
            
            # Đổi ngược trục y
            ax.invert_yaxis()
            
            # Tô màu các ô
            for row in range(size):
                for col in range(size):
                    val = grid[row, col]
                    if val != 0:  # Bỏ qua ô trống
                        # Tính màu dựa trên giá trị
                        color = cmap(val / (size**2 - 1))
                        rect = plt.Rectangle((col, row), 1, 1, facecolor=color, alpha=0.7, edgecolor='black')
                        ax.add_patch(rect)
                        
                        # Vẽ số trong các ô
                        ax.text(col + 0.5, row + 0.5, str(val), va='center', ha='center', 
                               fontsize=18, fontweight='bold', color='white')
            
            # Tô màu ô trống
            blank_row, blank_col = np.where(grid == 0)
            if len(blank_row) > 0 and len(blank_col) > 0:
                blank_row, blank_col = blank_row[0], blank_col[0]
                rect = plt.Rectangle((blank_col, blank_row), 1, 1, facecolor='lightgray', 
                                   alpha=0.5, edgecolor='black', linewidth=2, linestyle='--')
                ax.add_patch(rect)
            
            # Thêm số bước và mũi tên chỉ hướng di chuyển
            if i < len(path) - 1:
                # Tìm ô đã di chuyển giữa hai trạng thái
                current_state = np.array(state).reshape(size, size)
                next_state = np.array(path[i+1]).reshape(size, size)
                
                # Tìm vị trí khác nhau giữa hai trạng thái
                diff_positions = np.where(current_state != next_state)
                if len(diff_positions[0]) >= 2:  # Cần ít nhất 2 vị trí khác nhau
                    # Vị trí đầu tiên khác nhau (thường là vị trí ô trống hiện tại)
                    row1, col1 = diff_positions[0][0], diff_positions[1][0]
                    # Vị trí thứ hai khác nhau
                    row2, col2 = diff_positions[0][1], diff_positions[1][1]
                    
                    # Vẽ mũi tên chỉ hướng di chuyển
                    dx = col2 - col1
                    dy = row2 - row1
                    if abs(dx) + abs(dy) == 1:  # Đảm bảo chỉ vẽ mũi tên giữa các ô kề nhau
                        ax.arrow(col1 + 0.5, row1 + 0.5, dx * 0.4, dy * 0.4, head_width=0.2, 
                               head_length=0.2, fc='red', ec='red', linewidth=2)
            
            ax.set_title(f"Bước {i}", fontsize=14, fontweight='bold')
            ax.set_aspect('equal')
    
    # Ẩn các axes thừa
    for i in range(len(path), len(axes)):
        axes[i].axis('off')
    
    # Thêm tiêu đề tổng thể
    plt.suptitle(f"Đường đi từ trạng thái ban đầu đến trạng thái đích ({len(path)} bước)", 
               fontsize=16, fontweight='bold', y=0.98)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Điều chỉnh để không đè lên tiêu đề
    return fig

In [155]:
def run_experiment(puzzle_data, algorithm, algorithm_name, max_puzzles=10, time_limit=60):
    results = []
    
    # Giới hạn số lượng puzzle để thử nghiệm
    puzzles_to_test = puzzle_data[:max_puzzles]
    
    for i, puzzle_info in enumerate(puzzles_to_test):
        print(f"Đang giải puzzle {i+1}/{len(puzzles_to_test)} với thuật toán {algorithm_name}...")
        
        # Tạo puzzle từ dữ liệu
        initial_state = puzzle_info["initial_state"]
        goal_state = puzzle_info["goal_state"]
        size = int(len(initial_state) ** 0.5)
        puzzle = Puzzle(initial_state, goal_state, size)
        
        # Đo thời gian thực thi
        start_time = time.time()
        
        # Truyền time_limit trực tiếp vào thuật toán thay vì sử dụng threading
        try:
            # Chạy thuật toán với time_limit
            solution, stats = algorithm(puzzle, time_limit=time_limit)
            timeout = stats.get("timeout", False)
        except Exception as e:
            print(f"Lỗi khi chạy thuật toán: {e}")
            solution = None
            stats = {"nodes_explored": 0, "max_memory": 0, "solution_length": 0, "timeout": True}
            timeout = True
        
        # Tính thời gian thực thi
        execution_time = time.time() - start_time
        
        # Nếu timeout, giới hạn thời gian thực thi báo cáo
        if timeout:
            execution_time = min(execution_time, time_limit)
            print(f"  Timeout sau {execution_time:.2f} giây!")
        
        # Lưu kết quả
        result = {
            "puzzle_id": puzzle_info["id"],
            "algorithm": algorithm_name,
            "solution_found": solution is not None,
            "solution_length": stats.get("solution_length", 0) if solution else 0,
            "nodes_explored": stats.get("nodes_explored", 0),
            "max_memory": stats.get("max_memory", 0),
            "execution_time": execution_time,
            "timeout": timeout
        }
        
        # Thêm các thống kê bổ sung nếu có
        for key, value in stats.items():
            if key not in ["nodes_explored", "max_memory", "solution_length", "timeout"]:
                result[key] = value
        
        results.append(result)
        
        # In thông tin nhanh về kết quả
        status = "Thành công" if solution else "Thất bại"
        print(f"  Kết quả: {status}, Nút thăm: {stats['nodes_explored']}, Thời gian: {execution_time:.2f}s")
    
    return results

In [156]:
def run_all_algorithms_8puzzle(puzzle_data, max_puzzles=10):
    all_results = {}
    
    # Danh sách các thuật toán và tên tương ứng - Đảm bảo mỗi thuật toán chấp nhận time_limit
    algorithms = [
        (lambda p, time_limit=60: breadth_first_search(p, time_limit), "BFS"),
        (lambda p, time_limit=60: depth_first_search(p, max_depth=100, time_limit=time_limit), "DFS"),
        (lambda p, time_limit=60: uniform_cost_search(p, time_limit), "Uniform-Cost Search"),
        (lambda p, time_limit=60: depth_limited_search(p, 30, time_limit), "Depth-Limited Search (limit=30)"),
        (lambda p, time_limit=60: iterative_deepening_search(p, 30, time_limit), "Iterative Deepening Search (max=30)"),
        (lambda p, time_limit=60: bidirectional_search(p, time_limit), "Bidirectional Search")
    ]
    
    # Tính độ khó của các puzzle một lần duy nhất
    puzzle_difficulty = {}
    for idx, puzzle_info in enumerate(puzzle_data[:max_puzzles]):
        initial_state = puzzle_info["initial_state"]
        goal_state = puzzle_info["goal_state"]
        
        # Đếm số ô sai vị trí
        difficulty = sum(1 for i in range(len(initial_state)) 
                       if initial_state[i] != 0 and initial_state[i] != goal_state[i])
        puzzle_difficulty[idx] = difficulty
    
    # Sắp xếp puzzle theo độ khó tăng dần để chạy hiệu quả
    sorted_indices = sorted(list(range(min(max_puzzles, len(puzzle_data)))), 
                          key=lambda idx: puzzle_difficulty[idx])
    
    # Đặt timeout dựa trên độ khó
    timeouts = {}
    for i, idx in enumerate(sorted_indices):
        if i < len(sorted_indices) / 3:
            timeouts[idx] = 15  # 15 giây cho puzzle dễ
        elif i < 2 * len(sorted_indices) / 3:
            timeouts[idx] = 30  # 30 giây cho puzzle trung bình
        else:
            timeouts[idx] = 60  # 60 giây cho puzzle khó
    
    # Chạy từng thuật toán
    for algorithm, name in algorithms:
        print(f"\nĐang chạy thuật toán: {name}")
        
        results = []
        for i, puzzle_info in enumerate(puzzle_data[:max_puzzles]):
            timeout = timeouts.get(i, 30)  # Mặc định 30 giây
            print(f"Đang giải puzzle {i+1}/{min(max_puzzles, len(puzzle_data))} với thuật toán {name} (timeout: {timeout}s)...")
            
            try:
                # Tạo puzzle từ dữ liệu
                initial_state = puzzle_info["initial_state"]
                goal_state = puzzle_info["goal_state"]
                size = int(len(initial_state) ** 0.5)
                puzzle = Puzzle(initial_state, goal_state, size)
                
                # Chạy thuật toán với timeout được chỉ định
                start_time = time.time()
                solution, stats = algorithm(puzzle, time_limit=timeout)
                execution_time = time.time() - start_time
                
                # Lưu kết quả
                result = {
                    "puzzle_id": puzzle_info["id"],
                    "algorithm": name,
                    "solution_found": solution is not None,
                    "solution_length": stats["solution_length"] if solution else 0,
                    "nodes_explored": stats["nodes_explored"],
                    "max_memory": stats["max_memory"],
                    "execution_time": min(execution_time, timeout),
                    "timeout": stats.get("timeout", False),
                    "difficulty": puzzle_difficulty.get(i, 0)
                }
                
                results.append(result)
                
                # In kết quả nhanh
                status = "Thành công" if solution else "Thất bại"
                print(f"  Kết quả: {status}, Nút thăm: {stats['nodes_explored']}, Thời gian: {execution_time:.2f}s")
                
            except Exception as e:
                print(f"Lỗi khi chạy thuật toán {name} trên puzzle {i+1}: {e}")
                results.append({
                    "puzzle_id": puzzle_info["id"],
                    "algorithm": name,
                    "solution_found": False,
                    "solution_length": 0,
                    "nodes_explored": 0,
                    "max_memory": 0,
                    "execution_time": timeout,
                    "timeout": True,
                    "difficulty": puzzle_difficulty.get(i, 0),
                    "error": str(e)
                })
        
        all_results[name] = results
    
    return all_results

In [157]:
def run_all_algorithms_15puzzle(puzzle_data, max_puzzles=5):
    all_results = {}
    
    # Tối ưu: Giảm số lượng thuật toán và sử dụng timeout ngắn hơn
    algorithms = [
        (lambda p, time_limit=60: breadth_first_search(p, time_limit), "BFS"),
        (lambda p, time_limit=60: depth_limited_search(p, 10, time_limit), "Depth-Limited Search (limit=10)"),
        (lambda p, time_limit=60: iterative_deepening_search(p, 10, time_limit), "Iterative Deepening Search (max=10)"),
        (lambda p, time_limit=60: bidirectional_search(p, time_limit), "Bidirectional Search")
    ]
    
    # Tính độ khó puzzle một lần - tối ưu
    puzzle_difficulty = {}
    for idx, puzzle_info in enumerate(puzzle_data[:max_puzzles]):
        initial_state = puzzle_info["initial_state"]
        goal_state = puzzle_info["goal_state"]
        
        # Đếm số ô sai vị trí
        difficulty = sum(1 for i in range(len(initial_state)) 
                      if initial_state[i] != 0 and initial_state[i] != goal_state[i])
        puzzle_difficulty[idx] = difficulty
    
    # Sắp xếp puzzle theo độ khó tăng dần
    sorted_indices = sorted(list(range(min(max_puzzles, len(puzzle_data)))),
                          key=lambda idx: puzzle_difficulty[idx])
    
    # Đặt timeout ngắn hơn cho 15-puzzle
    timeouts = {}
    for i, idx in enumerate(sorted_indices):
        if i < len(sorted_indices) / 3:
            timeouts[idx] = 30  # 30 giây cho puzzle dễ
        elif i < 2 * len(sorted_indices) / 3:
            timeouts[idx] = 60  # 60 giây cho puzzle trung bình
        else:
            timeouts[idx] = 90  # 90 giây cho puzzle khó
    
    # Chạy từng thuật toán
    for algorithm, name in algorithms:
        print(f"\nĐang chạy thuật toán: {name}")
        
        results = []
        for i, puzzle_info in enumerate(puzzle_data[:max_puzzles]):
            timeout = timeouts.get(i, 60)  # Mặc định 60 giây
            print(f"Đang giải puzzle {i+1}/{min(max_puzzles, len(puzzle_data))} với thuật toán {name} (timeout: {timeout}s)...")
            
            try:
                # Tạo puzzle
                initial_state = puzzle_info["initial_state"]
                goal_state = puzzle_info["goal_state"]
                size = int(len(initial_state) ** 0.5)  # 4 cho 15-puzzle
                puzzle = Puzzle(initial_state, goal_state, size)
                
                # Chạy thuật toán
                start_time = time.time()
                solution, stats = algorithm(puzzle, time_limit=timeout)
                execution_time = time.time() - start_time
                
                # Lưu kết quả
                result = {
                    "puzzle_id": puzzle_info["id"],
                    "algorithm": name,
                    "solution_found": solution is not None,
                    "solution_length": stats["solution_length"] if solution else 0,
                    "nodes_explored": stats["nodes_explored"],
                    "max_memory": stats["max_memory"],
                    "execution_time": min(execution_time, timeout),
                    "timeout": stats.get("timeout", False),
                    "difficulty": puzzle_difficulty.get(i, 0)
                }
                
                results.append(result)
                
                # In kết quả nhanh
                status = "Thành công" if solution else "Thất bại"
                print(f"  Kết quả: {status}, Nút thăm: {stats['nodes_explored']}, Thời gian: {execution_time:.2f}s")
                
            except Exception as e:
                print(f"Lỗi khi chạy thuật toán {name} trên puzzle {i+1}: {e}")
                results.append({
                    "puzzle_id": puzzle_info["id"],
                    "algorithm": name,
                    "solution_found": False,
                    "solution_length": 0,
                    "nodes_explored": 0,
                    "max_memory": 0,
                    "execution_time": timeout,
                    "timeout": True,
                    "difficulty": puzzle_difficulty.get(i, 0),
                    "error": str(e)
                })
        
        all_results[name] = results
    
    return all_results

In [158]:
def evaluate_performance(results):
    """
    Đánh giá hiệu suất của các thuật toán với trọng tâm vào:
    - Số lượng nút khám phá (explored nodes)
    - Thời gian thực thi (execution time)
    - Chất lượng lời giải (solution quality)
    """
    performance = []
    
    for algorithm_name, algorithm_results in results.items():
        # Số lượng puzzle đã giải thành công
        solved_puzzles = sum(1 for r in algorithm_results if r["solution_found"])
        total_puzzles = len(algorithm_results)
        success_rate = solved_puzzles / total_puzzles if total_puzzles > 0 else 0
        
        # Số lượng nút khám phá
        nodes_explored = [r["nodes_explored"] for r in algorithm_results]
        avg_nodes_explored = np.mean(nodes_explored) if nodes_explored else 0
        max_nodes_explored = max(nodes_explored) if nodes_explored else 0
        min_nodes_explored = min([n for n in nodes_explored if n > 0]) if [n for n in nodes_explored if n > 0] else 0
        
        # Thời gian thực thi
        execution_times = [r["execution_time"] for r in algorithm_results]
        avg_execution_time = np.mean(execution_times) if execution_times else 0
        max_execution_time = max(execution_times) if execution_times else 0
        min_execution_time = min([t for t in execution_times if t > 0]) if [t for t in execution_times if t > 0] else 0
        
        # Chất lượng lời giải
        solution_lengths = [r["solution_length"] for r in algorithm_results if r["solution_found"]]
        avg_solution_length = np.mean(solution_lengths) if solution_lengths else 0
        solution_length_variance = np.var(solution_lengths) if len(solution_lengths) > 1 else 0
        
        # Tỷ lệ nút thăm / độ dài lời giải - chỉ số hiệu quả tìm kiếm
        if solved_puzzles > 0:
            search_efficiency_values = [
                r["nodes_explored"] / r["solution_length"] 
                for r in algorithm_results if r["solution_found"] and r["solution_length"] > 0
            ]
            search_efficiency = np.mean(search_efficiency_values) if search_efficiency_values else float('inf')
        else:
            search_efficiency = float('inf')
        
        # Tỷ lệ timeout
        timeout_rate = sum(1 for r in algorithm_results if r.get("timeout", False)) / total_puzzles if total_puzzles > 0 else 0
        
        # Lưu thông tin hiệu suất
        performance.append({
            "Algorithm": algorithm_name,
            "Success Rate": success_rate,
            
            # Explored Nodes
            "Avg. Nodes Explored": avg_nodes_explored,
            "Max Nodes Explored": max_nodes_explored,
            "Min Nodes Explored": min_nodes_explored,
            
            # Execution Time
            "Avg. Execution Time (s)": avg_execution_time,
            "Max Execution Time (s)": max_execution_time,
            "Min Execution Time (s)": min_execution_time,
            
            # Solution Quality
            "Avg. Solution Length": avg_solution_length,
            "Solution Length Variance": solution_length_variance,
            "Search Efficiency (nodes/step)": search_efficiency,
            
            # Additional
            "Timeout Rate": timeout_rate
        })
    
    # Tạo DataFrame
    performance_df = pd.DataFrame(performance)
    
    # Sắp xếp kết quả
    if not performance_df.empty:
        if performance_df["Success Rate"].sum() > 0:
            performance_df = performance_df.sort_values(
                by=["Success Rate", "Avg. Execution Time (s)"], 
                ascending=[False, True]
            )
        else:
            performance_df = performance_df.sort_values(
                by="Avg. Execution Time (s)", 
                ascending=True
            )
    
    return performance_df

In [159]:
# Hàm vẽ biểu đồ so sánh
def plot_comparison(performance_df, metric, title, ylabel, figsize=(12, 8)):
    """
    Vẽ biểu đồ so sánh hiệu suất của các thuật toán
    
    Args:
        performance_df (DataFrame): Bảng hiệu suất
        metric (str): Tên cột cần vẽ
        title (str): Tiêu đề biểu đồ
        ylabel (str): Nhãn trục y
        figsize (tuple): Kích thước biểu đồ
    """
    plt.figure(figsize=figsize)
    
    # Tạo biểu đồ cột
    bars = plt.bar(performance_df["Algorithm"], performance_df[metric])
    
    # Thêm nhãn giá trị
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{height:.2f}', ha='center', va='bottom')
    
    plt.title(title, fontsize=14)
    plt.xlabel("Thuật toán", fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    
    return plt.gcf()


In [160]:
def plot_all_metrics(performance_df):
    """
    Vẽ biểu đồ so sánh tất cả các chỉ số hiệu suất - tối ưu
    """
    # Nếu DataFrame rỗng, không vẽ biểu đồ
    if performance_df.empty:
        return []
    
    # Cấu hình màu sắc và style cho đồ thị
    plt.style.use('seaborn-v0_8-darkgrid')
    colors = plt.cm.viridis(np.linspace(0, 1, len(performance_df)))
    
    # Danh sách chỉ số cần vẽ biểu đồ
    basic_metrics = [
        ("Success Rate", "Tỷ lệ thành công", "Tỷ lệ"),
        ("Avg. Nodes Explored", "Số nút đã khám phá trung bình", "Số nút"),
        ("Avg. Execution Time (s)", "Thời gian thực thi trung bình", "Thời gian (giây)")
    ]
    
    # Thêm độ dài lời giải nếu có
    if "Avg. Solution Length" in performance_df.columns:
        basic_metrics.append((
            "Avg. Solution Length", 
            "Độ dài trung bình của lời giải", 
            "Số bước di chuyển"
        ))
    
    # Danh sách chỉ số hiệu quả
    efficiency_metrics = []
    if "Search Efficiency (nodes/step)" in performance_df.columns:
        efficiency_metrics.append((
            "Search Efficiency (nodes/step)", 
            "Hiệu quả tìm kiếm", 
            "Nút/bước"
        ))
    
    # Kết hợp tất cả các chỉ số
    all_metrics = basic_metrics + efficiency_metrics
    
    figures = []
    
    # Tối ưu: Sử dụng multiprocessing để tạo các biểu đồ song song
    for metric, title, ylabel in all_metrics:
        if metric not in performance_df.columns:
            continue
            
        fig, ax = plt.subplots(figsize=(10, 6))  # Giảm kích thước để tạo nhanh hơn
        
        # Sắp xếp theo metric
        sorted_df = performance_df.sort_values(by=metric)
        
        # Quyết định sử dụng log scale nếu giá trị lớn
        use_log = metric in ["Avg. Nodes Explored", "Avg. Max Memory", "Search Efficiency (nodes/step)"]
        
        # Tạo biểu đồ cột
        bars = ax.bar(sorted_df["Algorithm"], sorted_df[metric], color=colors)
        
        # Thêm nhãn giá trị
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                # Format nhãn
                if height < 0.01:
                    value_text = f'{height:.1e}'
                elif height < 1:
                    value_text = f'{height:.3f}'
                elif height < 10:
                    value_text = f'{height:.2f}'
                elif height < 1000:
                    value_text = f'{height:.1f}'
                else:
                    value_text = f'{height:.2e}'
                
                ax.text(bar.get_x() + bar.get_width()/2., height * 1.02,
                       value_text, ha='center', va='bottom', rotation=0, fontsize=9)
        
        # Thêm nhãn và tiêu đề
        ax.set_title(title, fontsize=14)
        ax.set_xlabel("Thuật toán", fontsize=11)
        ax.set_ylabel(ylabel, fontsize=11)
        
        # Xử lý trục y (log scale nếu cần)
        if use_log and sorted_df[metric].max() > 1000:
            ax.set_yscale('log')
            ax.set_ylabel(f"{ylabel} (log scale)", fontsize=11)
        
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        
        figures.append(fig)
    
    # Tạo biểu đồ radar cho so sánh tổng hợp nếu có đủ dữ liệu
    if len(performance_df) > 1:
        fig = plt.figure(figsize=(10, 8))  # Giảm kích thước
        
        # Chuẩn hóa dữ liệu cốt lõi
        normalized_df = performance_df.copy()
        metrics_to_normalize = {
            "Success Rate": True,  # True = giá trị cao là tốt
            "Avg. Nodes Explored": False,  # False = giá trị thấp là tốt
            "Avg. Execution Time (s)": False
        }
        
        for col, is_higher_better in metrics_to_normalize.items():
            if col in normalized_df.columns:
                min_val = normalized_df[col].min()
                max_val = normalized_df[col].max()
                
                if max_val > min_val:
                    if is_higher_better:
                        normalized_df[col] = (normalized_df[col] - min_val) / (max_val - min_val)
                    else:
                        normalized_df[col] = 1 - ((normalized_df[col] - min_val) / (max_val - min_val))
                else:
                    normalized_df[col] = 0.5
        
        # Tạo biểu đồ radar với các thông số quan trọng
        categories = ['Tỷ lệ thành công', 'Số nút thăm', 'Thời gian']
        
        N = len(categories)
        angles = [n / float(N) * 2 * np.pi for n in range(N)]
        angles += angles[:1]  # Đóng vòng tròn
        
        ax = fig.add_subplot(111, polar=True)
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        plt.xticks(angles[:-1], categories)
        
        # Vẽ mỗi thuật toán
        for i, alg in enumerate(normalized_df["Algorithm"]):
            values = [
                normalized_df.loc[normalized_df["Algorithm"] == alg, "Success Rate"].values[0],
                normalized_df.loc[normalized_df["Algorithm"] == alg, "Avg. Nodes Explored"].values[0],
                normalized_df.loc[normalized_df["Algorithm"] == alg, "Avg. Execution Time (s)"].values[0]
            ]
            values += values[:1]  # Đóng vòng tròn
            
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=alg, color=colors[i])
            ax.fill(angles, values, alpha=0.1, color=colors[i])
        
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.title("So sánh hiệu suất các thuật toán", fontsize=14, y=1.1)
        
        figures.append(fig)
    
    return figures

In [161]:
def save_figures(figures, prefix):
    """
    Lưu biểu đồ - tối ưu
    """
    # Tạo thư mục lưu biểu đồ nếu chưa tồn tại
    if not os.path.exists('figures'):
        os.makedirs('figures')
    
    # Lưu từng biểu đồ
    for i, fig in enumerate(figures):
        try:
            filename = f"figures/{prefix}_metric_{i+1}.png"
            fig.savefig(filename, dpi=100)  # Giảm DPI để tạo nhanh hơn
            plt.close(fig)
        except Exception as e:
            print(f"Lỗi khi lưu biểu đồ {i+1}: {e}")
    
    print(f"Đã lưu {len(figures)} biểu đồ vào thư mục figures/")

In [162]:
def visualize_results():
    """
    Trực quan hóa kết quả thí nghiệm - tối ưu
    """
    # Đọc kết quả từ file CSV
    try:
        performance_8puzzle = pd.read_csv("results/performance_uninformed_8puzzle_all_datasets.csv")
    except:
        # Nếu file tổng hợp chưa có, thử đọc từng file riêng lẻ và gộp lại
        performance_files = []
        for size in [10, 100]:  # Chỉ lấy 2 kích thước
            try:
                df = pd.read_csv(f"results/performance_uninformed_8puzzle_{size}.csv")
                performance_files.append(df)
            except:
                print(f"Không tìm thấy file kết quả cho kích thước {size}")
        
        if performance_files:
            performance_8puzzle = pd.concat(performance_files)
        else:
            print("Không tìm thấy file kết quả nào!")
            return
    
    # Cũng thử đọc kết quả 15-puzzle nếu có
    try:
        performance_15puzzle = pd.read_csv("results/performance_15puzzle_all_datasets.csv")
        has_15puzzle_data = True
    except:
        has_15puzzle_data = False
    
    # Để tăng tốc, chỉ vẽ một vài biểu đồ quan trọng nhất
    # Tối ưu: Không vẽ tất cả biểu đồ từ đầu
    create_all_charts = False  # Thay đổi thành True nếu muốn vẽ tất cả biểu đồ
    
    if create_all_charts:
        # Vẽ biểu đồ quan trọng
        plt.figure(figsize=(10, 6))
        performance_sorted = performance_8puzzle.sort_values("Avg. Nodes Explored")
        bars = plt.bar(performance_sorted["Algorithm"], performance_sorted["Avg. Nodes Explored"], 
                     color=plt.cm.viridis(np.linspace(0, 1, len(performance_sorted))))
        
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.0f}', ha='center', va='bottom', fontsize=9)
        
        plt.title("SO SÁNH SỐ LƯỢNG NÚT KHÁM PHÁ GIỮA CÁC THUẬT TOÁN", fontsize=14)
        plt.xlabel("Thuật toán", fontsize=12)
        plt.ylabel("Số nút khám phá trung bình", fontsize=12)
        plt.xticks(rotation=45, ha="right")
        
        if performance_sorted["Avg. Nodes Explored"].max() > 1000:
            plt.yscale('log')
            plt.ylabel("Số nút khám phá trung bình (log scale)", fontsize=12)
        
        plt.tight_layout()
        plt.savefig("figures/explored_nodes_comparison.png")
        plt.close()
        
        # So sánh thời gian thực thi
        plt.figure(figsize=(10, 6))
        performance_sorted = performance_8puzzle.sort_values("Avg. Execution Time (s)")
        bars = plt.bar(performance_sorted["Algorithm"], performance_sorted["Avg. Execution Time (s)"],
                     color=plt.cm.viridis(np.linspace(0, 1, len(performance_sorted))))
        
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}s', ha='center', va='bottom', fontsize=9)
        
        plt.title("SO SÁNH THỜI GIAN THỰC THI GIỮA CÁC THUẬT TOÁN", fontsize=14)
        plt.xlabel("Thuật toán", fontsize=12)
        plt.ylabel("Thời gian thực thi trung bình (giây)", fontsize=12)
        plt.xticks(rotation=45, ha="right")
        
        plt.tight_layout()
        plt.savefig("figures/execution_time_comparison.png")
        plt.close()
    
    print("\nHoàn thành trực quan hóa kết quả!")

In [163]:
def show_aggregate_statistics(all_results):
    """
    Hiển thị thống kê tổng hợp từ tất cả kết quả - tối ưu
    """
    # Top 3 thuật toán tốt nhất dựa trên tỷ lệ thành công
    top_algorithms = all_results.sort_values(by=["Success Rate", "Avg. Execution Time (s)"], 
                                         ascending=[False, True]).head(3)
    
    print("\nTop 3 thuật toán có tỷ lệ thành công cao nhất:")
    print(top_algorithms[["Algorithm", "Success Rate", "Avg. Nodes Explored", "Avg. Execution Time (s)", "Avg. Solution Length"]])
    
    # Phân tích hiệu quả theo 3 khía cạnh chính
    print("\n=== THỐNG KÊ CHI TIẾT THEO TRỌNG TÂM ĐÁNH GIÁ ===")
    
    # 1. Phân tích số lượng nút đã khám phá
    print("\n1. PHÂN TÍCH SỐ LƯỢNG NÚT KHÁM PHÁ:")
    nodes_stats = all_results.sort_values(by="Avg. Nodes Explored")
    print(f"- Thuật toán khám phá ít nút nhất: {nodes_stats.iloc[0]['Algorithm']} (trung bình {nodes_stats.iloc[0]['Avg. Nodes Explored']:.1f} nút)")
    print(f"- Thuật toán khám phá nhiều nút nhất: {nodes_stats.iloc[-1]['Algorithm']} (trung bình {nodes_stats.iloc[-1]['Avg. Nodes Explored']:.1f} nút)")
    
    # 2. Phân tích thời gian thực thi
    print("\n2. PHÂN TÍCH THỜI GIAN THỰC THI:")
    time_stats = all_results.sort_values(by="Avg. Execution Time (s)")
    print(f"- Thuật toán nhanh nhất: {time_stats.iloc[0]['Algorithm']} (trung bình {time_stats.iloc[0]['Avg. Execution Time (s)']:.2f} giây)")
    print(f"- Thuật toán chậm nhất: {time_stats.iloc[-1]['Algorithm']} (trung bình {time_stats.iloc[-1]['Avg. Execution Time (s)']:.2f} giây)")
    
    # 3. Phân tích chất lượng lời giải
    print("\n3. PHÂN TÍCH CHẤT LƯỢNG LỜI GIẢI:")
    # Lọc chỉ thuật toán có lời giải (Avg. Solution Length > 0)
    solution_quality = all_results[all_results["Avg. Solution Length"] > 0].sort_values(by="Avg. Solution Length")
    if not solution_quality.empty:
        print(f"- Thuật toán có lời giải ngắn nhất: {solution_quality.iloc[0]['Algorithm']} (trung bình {solution_quality.iloc[0]['Avg. Solution Length']:.1f} bước)")
        
        # Phân tích hiệu quả tìm kiếm (nodes/step) - càng thấp càng tốt
        efficiency_stats = solution_quality.sort_values(by="Search Efficiency (nodes/step)")
        print(f"- Thuật toán hiệu quả nhất (nodes/step thấp nhất): {efficiency_stats.iloc[0]['Algorithm']} ({efficiency_stats.iloc[0]['Search Efficiency (nodes/step)']:.1f} nút/bước)")
        print(f"- Thuật toán kém hiệu quả nhất (nodes/step cao nhất): {efficiency_stats.iloc[-1]['Algorithm']} ({efficiency_stats.iloc[-1]['Search Efficiency (nodes/step)']:.1f} nút/bước)")
    else:
        print("Không có thuật toán nào tìm được lời giải!")
        
    # Không tạo biểu đồ để tiết kiệm thời gian

In [164]:
def save_results(performance_df, filename):
    """
    Lưu kết quả vào file CSV với thêm thông tin chi tiết
    
    Args:
        performance_df (DataFrame): Bảng hiệu suất
        filename (str): Tên file
    """
    # Thêm timestamp
    performance_df["Timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Xác định các cột cơ bản
    base_columns = ["Algorithm", "Success Rate", "Avg. Nodes Explored", "Avg. Execution Time (s)", "Timestamp"]
    
    # Thêm cột Algorithm Type và Puzzle nếu có
    if "Algorithm Type" in performance_df.columns:
        base_columns.insert(1, "Algorithm Type")
    
    if "Puzzle" in performance_df.columns:
        base_columns.insert(base_columns.index("Algorithm Type") + 1 if "Algorithm Type" in base_columns else 1, "Puzzle")
    
    if "Dataset Size" in performance_df.columns:
        base_columns.insert(base_columns.index("Puzzle") + 1 if "Puzzle" in base_columns else 
                         base_columns.index("Algorithm Type") + 1 if "Algorithm Type" in base_columns else 1, 
                         "Dataset Size")
    
    # Thêm cột Avg. Solution Length nếu có
    if "Avg. Solution Length" in performance_df.columns:
        base_columns.insert(base_columns.index("Avg. Execution Time (s)"), "Avg. Solution Length")
    
    # Thêm các cột bổ sung nếu có
    extra_columns = [col for col in performance_df.columns if col not in base_columns]
    column_order = base_columns + extra_columns
    
    # Lọc ra chỉ các cột tồn tại trong DataFrame
    existing_columns = [col for col in column_order if col in performance_df.columns]
    
    # Sắp xếp DataFrame theo thứ tự cột
    ordered_df = performance_df[existing_columns]
    
    # Tạo thư mục chứa nếu chưa tồn tại
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Lưu vào file CSV
    ordered_df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"Đã lưu kết quả vào file {filename}")
    
    # Thêm một phiên bản dạng pretty-formatted để dễ đọc (chú ý encoding)
    pretty_filename = filename.replace(".csv", "_prettified.txt")
    with open(pretty_filename, 'w', encoding='utf-8') as f:
        f.write(f"Ket qua thi nghiem: {os.path.basename(filename)}\\n")
        f.write(f"Thoi gian: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\\n")
        
        # Viết header
        header_format = "{:<30} {:<15} {:<15} {:<10} {:<10} {:<15} {:<15}\\n"
        f.write(header_format.format(
            "Thuat toan", "Loai thuat toan", "Puzzle", "Thanh cong", 
            "Do dai", "Nut tham", "Thoi gian (s)"
        ))
        
        f.write("-" * 120 + "\\n")
        
        # Viết dữ liệu
        row_format = "{:<30} {:<15} {:<15} {:<10.2f} {:<10.2f} {:<15.1f} {:<15.3f}\\n"
        for _, row in ordered_df.iterrows():
            try:
                f.write(row_format.format(
                    row["Algorithm"], 
                    row.get("Algorithm Type", ""), 
                    row.get("Puzzle", ""),
                    row["Success Rate"],
                    row.get("Avg. Solution Length", 0.0),
                    row["Avg. Nodes Explored"],
                    row["Avg. Execution Time (s)"]
                ))
            except Exception as e:
                # Nếu gặp lỗi khi ghi dữ liệu, chỉ ghi một phần dữ liệu cơ bản
                f.write(f"{row['Algorithm']}: Success Rate={row['Success Rate']:.2f}, "
                       f"Nodes={row['Avg. Nodes Explored']:.1f}, Time={row['Avg. Execution Time (s)']:.3f}s\n")
    
    print(f"Đã lưu phiên bản dễ đọc vào file {pretty_filename}")

In [165]:
def analyze_puzzle_difficulty(puzzle_data, max_puzzles, puzzle_type):
    """
    Phân tích và hiển thị độ khó của các puzzle - tối ưu
    """
    puzzle_metrics = []
    
    # Chỉ phân tích max_puzzles puzzle đầu tiên
    limited_puzzles = puzzle_data[:max_puzzles]
    
    # Tối ưu: Sử dụng list comprehension và vectorization với numpy
    initial_states = [puzzle_info["initial_state"] for puzzle_info in limited_puzzles]
    goal_states = [puzzle_info["goal_state"] for puzzle_info in limited_puzzles]
    
    size = int(len(initial_states[0]) ** 0.5) if initial_states else 0
    
    for i, (initial_state, goal_state) in enumerate(zip(initial_states, goal_states)):
        # Tính số ô sai vị trí (Misplaced Tiles)
        misplaced = sum(1 for j in range(len(initial_state)) 
                      if initial_state[j] != 0 and initial_state[j] != goal_state[j])
        
        # Ước lượng độ dài đường đi
        path_length_estimate = misplaced * 1.5
        
        # Thêm vào danh sách
        puzzle_metrics.append({
            "Puzzle ID": limited_puzzles[i]["id"],
            "Misplaced Tiles": misplaced,
            "Path Length Estimate": path_length_estimate,
            "Combined Score": (misplaced + path_length_estimate) / 2
        })
    
    # Tạo DataFrame
    metrics_df = pd.DataFrame(puzzle_metrics)
    
    # Sắp xếp theo độ khó tăng dần
    metrics_df = metrics_df.sort_values(by="Combined Score")
    
    # Lưu kết quả
    metrics_df.to_csv(f"results/puzzle_difficulty_{puzzle_type.replace('-', '')}.csv", index=False)
    
    # In kết quả
    print(f"\nĐộ khó của các {puzzle_type}:")
    print(metrics_df)
    
    # Chỉ tạo biểu đồ nếu cần thiết (để tăng tốc)
    create_difficulty_plot = False  # Thay đổi thành True nếu muốn tạo biểu đồ
    
    if create_difficulty_plot:
        fig, ax = plt.subplots(figsize=(8, 5))
        
        # Tạo heatmap đơn giản hơn
        im = ax.imshow(metrics_df[["Misplaced Tiles", "Path Length Estimate"]].values, cmap="YlOrRd")
        
        # Thêm nhãn
        ax.set_xticks(np.arange(2))
        ax.set_yticks(np.arange(len(metrics_df)))
        ax.set_xticklabels(["Misplaced Tiles", "Path Length Estimate"])
        ax.set_yticklabels([f"Puzzle {i}" for i in metrics_df["Puzzle ID"]])
        
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        
        # Thêm giá trị vào mỗi ô
        for i in range(len(metrics_df)):
            for j in range(2):
                ax.text(j, i, f"{metrics_df.iloc[i, j+1]:.1f}", ha="center", va="center", color="black")
        
        # Thêm colorbar
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel("Độ khó", rotation=-90, va="bottom")
        
        # Thêm tiêu đề
        ax.set_title(f"Độ khó của các {puzzle_type}")
        
        plt.tight_layout()
        plt.savefig(f"figures/puzzle_difficulty_{puzzle_type.replace('-', '')}.png")
        plt.close(fig)
    
    return metrics_df

In [166]:
def compare_puzzle_types(results_8puzzle, results_15puzzle):
    """
    So sánh hiệu suất giữa 8-puzzle và 15-puzzle - tối ưu
    """
    # Lọc ra các thuật toán có trong cả hai loại puzzle
    common_algorithms = set(results_8puzzle["Algorithm"]) & set(results_15puzzle["Algorithm"])
    
    if not common_algorithms:
        print("Không có thuật toán chung giữa hai loại puzzle để so sánh!")
        return
    
    # Lọc dữ liệu
    results_8puzzle_filtered = results_8puzzle[results_8puzzle["Algorithm"].isin(common_algorithms)]
    results_15puzzle_filtered = results_15puzzle[results_15puzzle["Algorithm"].isin(common_algorithms)]
    
    # Tạo DataFrame so sánh - sử dụng list comprehension
    comparison_data = [
        {
            "Algorithm": algorithm,
            "Success Rate (8-puzzle)": results_8puzzle_filtered[results_8puzzle_filtered["Algorithm"] == algorithm]["Success Rate"].values[0],
            "Success Rate (15-puzzle)": results_15puzzle_filtered[results_15puzzle_filtered["Algorithm"] == algorithm]["Success Rate"].values[0],
            "Avg. Nodes Explored (8-puzzle)": results_8puzzle_filtered[results_8puzzle_filtered["Algorithm"] == algorithm]["Avg. Nodes Explored"].values[0],
            "Avg. Nodes Explored (15-puzzle)": results_15puzzle_filtered[results_15puzzle_filtered["Algorithm"] == algorithm]["Avg. Nodes Explored"].values[0],
            "Avg. Execution Time (8-puzzle)": results_8puzzle_filtered[results_8puzzle_filtered["Algorithm"] == algorithm]["Avg. Execution Time (s)"].values[0],
            "Avg. Execution Time (15-puzzle)": results_15puzzle_filtered[results_15puzzle_filtered["Algorithm"] == algorithm]["Avg. Execution Time (s)"].values[0]
        }
        for algorithm in common_algorithms
    ]
    
    # Thêm tỷ lệ 
    for item in comparison_data:
        # Success Rate Ratio
        item["Success Rate Ratio"] = item["Success Rate (8-puzzle)"] / max(item["Success Rate (15-puzzle)"], 0.001)
        
        # Nodes Explored Ratio
        item["Nodes Explored Ratio"] = item["Avg. Nodes Explored (15-puzzle)"] / max(item["Avg. Nodes Explored (8-puzzle)"], 1)
        
        # Execution Time Ratio
        item["Execution Time Ratio"] = item["Avg. Execution Time (15-puzzle)"] / max(item["Avg. Execution Time (8-puzzle)"], 0.001)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # In kết quả so sánh
    print("\nSo sánh hiệu suất giữa 8-puzzle và 15-puzzle:")
    print(comparison_df[["Algorithm", "Success Rate (8-puzzle)", "Success Rate (15-puzzle)"]])
    
    # Lưu kết quả so sánh
    comparison_df.to_csv("results/comparison_8puzzle_vs_15puzzle.csv", index=False)
    
    # Chỉ tạo biểu đồ cần thiết (để tăng tốc)
    create_comparison_plots = False  # Thay đổi thành True nếu muốn tạo tất cả biểu đồ
    
    if create_comparison_plots:
        # Tạo biểu đồ cột so sánh tỷ lệ thành công
        plt.figure(figsize=(10, 6))
        algorithms = comparison_df["Algorithm"]
        x = np.arange(len(algorithms))
        width = 0.35
        
        fig, ax = plt.subplots(figsize=(10, 6))
        rects1 = ax.bar(x - width/2, comparison_df["Success Rate (8-puzzle)"], width, label='8-puzzle')
        rects2 = ax.bar(x + width/2, comparison_df["Success Rate (15-puzzle)"], width, label='15-puzzle')
        
        ax.set_xlabel('Thuật toán')
        ax.set_ylabel('Tỷ lệ thành công')
        ax.set_title('So sánh tỷ lệ thành công giữa 8-puzzle và 15-puzzle')
        ax.set_xticks(x)
        ax.set_xticklabels(algorithms, rotation=45, ha='right')
        ax.legend()
        
        plt.tight_layout()
        plt.savefig("figures/comparison_success_rate.png")
        plt.close()
        
        # Tạo biểu đồ radar
        plt.figure(figsize=(8, 8))
        
        # Chuẩn hóa dữ liệu
        success_rate_normalized = (comparison_df["Success Rate (8-puzzle)"] / comparison_df["Success Rate (8-puzzle)"].max())
        
        nodes_ratio_normalized = 1 - ((comparison_df["Nodes Explored Ratio"] - comparison_df["Nodes Explored Ratio"].min()) / 
                                   (comparison_df["Nodes Explored Ratio"].max() - comparison_df["Nodes Explored Ratio"].min() + 0.001))
        
        time_ratio_normalized = 1 - ((comparison_df["Execution Time Ratio"] - comparison_df["Execution Time Ratio"].min()) / 
                                  (comparison_df["Execution Time Ratio"].max() - comparison_df["Execution Time Ratio"].min() + 0.001))
        
        categories = ['Tỷ lệ thành công\n(8-puzzle)', 'Tỷ lệ nút khám phá\n(8/15-puzzle)', 'Tỷ lệ thời gian\n(8/15-puzzle)']
        N = len(categories)
        
        angles = [n / float(N) * 2 * np.pi for n in range(N)]
        angles += angles[:1]
        
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
        
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        
        plt.xticks(angles[:-1], categories)
        
        colors = plt.cm.viridis(np.linspace(0, 1, len(algorithms)))
        
        for i, algorithm in enumerate(algorithms):
            values = [
                success_rate_normalized.iloc[i],
                nodes_ratio_normalized.iloc[i],
                time_ratio_normalized.iloc[i]
            ]
            
            values += values[:1]
            
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=algorithm, color=colors[i])
            ax.fill(angles, values, alpha=0.1, color=colors[i])
        
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        
        plt.title("So sánh hiệu suất các thuật toán giữa 8-puzzle và 15-puzzle", fontsize=14, y=1.1)
        
        plt.savefig("figures/radar_comparison_puzzle_types.png")
        plt.close()
        
    return comparison_df

In [167]:
def comprehensive_main():
    """
    Hàm chạy toàn bộ thí nghiệm với tất cả các thuật toán không thông tin
    trên tất cả các bộ dữ liệu (10, 100, 1000, 10000) để so sánh hiệu suất.
    Tập trung đánh giá vào:
    - Số lượng nút khám phá (explored nodes)
    - Thời gian thực thi (execution time)
    - Chất lượng lời giải (solution quality)
    """
    # Tạo thư mục lưu kết quả nếu chưa tồn tại
    for directory in ['results', 'figures']:
        if not os.path.exists(directory):
            os.makedirs(directory)
    
    # Danh sách kích thước dataset
    dataset_sizes = [10, 100, 1000, 10000]
    
    # Danh sách để lưu kết quả cho mỗi loại puzzle và kích thước dataset
    all_results_8puzzle = []
    all_results_15puzzle = []
    
    # Theo dõi thời gian chạy tổng thể
    start_time_total = time.time()
    
    # Chạy thí nghiệm cho từng kích thước dataset
    for size in dataset_sizes:
        print(f"\n==== Thí nghiệm với bộ dữ liệu kích thước {size} dataset ====")
        
        # Theo dõi thời gian chạy cho dataset này
        start_time_dataset = time.time()
        
        # 1. Thí nghiệm với 8-puzzle
        print(f"\n=== Thí nghiệm với 8-puzzle (kích thước {size} dataset) ===")
        
        try:
            # Đọc dữ liệu
            puzzle_8_data = load_puzzle_data(f"data8puzzle/dataset_{size}.json")
            
            # Giới hạn số lượng puzzles để đảm bảo hiệu suất tốt với các bộ dữ liệu lớn
            max_puzzles = min(10, size)  # Với bộ dữ liệu lớn, chỉ chạy một số lượng nhỏ
            if size >= 1000:
                max_puzzles = 5  # Giảm số lượng puzzles với bộ dữ liệu rất lớn
            
            # Chạy tất cả các thuật toán không thông tin
            print("\nChạy các thuật toán không thông tin...")
            results_uninformed_8puzzle = run_all_algorithms_8puzzle(puzzle_8_data, max_puzzles=max_puzzles)
            
            # Đánh giá hiệu suất
            performance_uninformed_8puzzle = evaluate_performance(results_uninformed_8puzzle)
            performance_uninformed_8puzzle["Algorithm Type"] = "Uninformed"
            performance_uninformed_8puzzle["Puzzle"] = "8-puzzle"
            performance_uninformed_8puzzle["Dataset Size"] = size
            
            # Lưu kết quả
            save_results(performance_uninformed_8puzzle, f"results/performance_uninformed_8puzzle_{size}.csv")
            
            # Tạo biểu đồ so sánh
            figures = plot_all_metrics(performance_uninformed_8puzzle)
            save_figures(figures, f"8puzzle_{size}")
            
            # Hợp nhất kết quả
            all_results_8puzzle.append(performance_uninformed_8puzzle)
            
            # Phân tích độ khó của các puzzle
            analyze_puzzle_difficulty(puzzle_8_data, max_puzzles, "8-puzzle")
        except Exception as e:
            print(f"\nLỗi khi thực hiện thí nghiệm với 8-puzzle kích thước {size}: {e}")
        
        # 2. Thí nghiệm với 15-puzzle (chỉ với dataset kích thước nhỏ)
        if size <= 10000:  # Giới hạn chỉ chạy với dataset nhỏ do 15-puzzle phức tạp hơn nhiều
            print(f"\n=== Thí nghiệm với 15-puzzle (kích thước {size} dataset) ===")
            
            try:
                # Đọc dữ liệu
                puzzle_15_data = load_puzzle_data(f"data15puzzle/dataset_{size}.json")
                
                # Giới hạn số lượng puzzles thậm chí ít hơn với 15-puzzle
                max_puzzles_15 = min(3, size)  # Giới hạn số lượng là 5 hoặc ít hơn
                
                # Chạy tất cả các thuật toán không thông tin với timeout dài hơn
                print("\nChạy các thuật toán không thông tin...")
                results_uninformed_15puzzle = run_all_algorithms_15puzzle(puzzle_15_data, max_puzzles=max_puzzles_15)
                
                # Đánh giá hiệu suất
                performance_uninformed_15puzzle = evaluate_performance(results_uninformed_15puzzle)
                performance_uninformed_15puzzle["Algorithm Type"] = "Uninformed"
                performance_uninformed_15puzzle["Puzzle"] = "15-puzzle"
                performance_uninformed_15puzzle["Dataset Size"] = size
                
                # Lưu kết quả
                save_results(performance_uninformed_15puzzle, f"results/performance_uninformed_15puzzle_{size}.csv")
                
                # Tạo biểu đồ so sánh
                figures = plot_all_metrics(performance_uninformed_15puzzle)
                save_figures(figures, f"15puzzle_{size}")
                
                # Hợp nhất kết quả
                all_results_15puzzle.append(performance_uninformed_15puzzle)
                
                # Phân tích độ khó của 15-puzzle
                analyze_puzzle_difficulty(puzzle_15_data, max_puzzles_15, "15-puzzle")
            except Exception as e:
                print(f"\nLỗi khi thực hiện thí nghiệm với 15-puzzle kích thước {size}: {e}")
        
        # In thời gian chạy cho dataset này
        dataset_time = time.time() - start_time_dataset
        print(f"\nThời gian chạy cho dataset kích thước {size}: {dataset_time:.2f} giây")
    
    # Hợp nhất tất cả kết quả cho 8-puzzle
    if all_results_8puzzle:
        all_results_8puzzle_combined = pd.concat(all_results_8puzzle)
        save_results(all_results_8puzzle_combined, "results/performance_8puzzle_all_datasets.csv")
        
        # Hiển thị thống kê tổng hợp cho 8-puzzle
        print("\n=== Thống kê tổng hợp cho 8-puzzle ===")
        show_aggregate_statistics(all_results_8puzzle_combined)
    
    # Hợp nhất tất cả kết quả cho 15-puzzle nếu có
    if all_results_15puzzle:
        all_results_15puzzle_combined = pd.concat(all_results_15puzzle)
        save_results(all_results_15puzzle_combined, "results/performance_15puzzle_all_datasets.csv")
        
        # Hiển thị thống kê tổng hợp cho 15-puzzle
        print("\n=== Thống kê tổng hợp cho 15-puzzle ===")
        show_aggregate_statistics(all_results_15puzzle_combined)
        
        # Hợp nhất kết quả từ cả hai loại puzzle
        all_results_combined = pd.concat([all_results_8puzzle_combined, all_results_15puzzle_combined])
        save_results(all_results_combined, "results/performance_all_puzzles.csv")
        
        # So sánh hiệu suất giữa 8-puzzle và 15-puzzle
        try:
            compare_puzzle_types(all_results_8puzzle_combined, all_results_15puzzle_combined)
        except Exception as e:
            print(f"\nLỗi khi so sánh các loại puzzle: {e}")
    
    # Trực quan hóa kết quả tổng thể
    try:
        visualize_results()
    except Exception as e:
        print(f"\nLỗi khi trực quan hóa kết quả: {e}")
    
    # In thời gian chạy tổng thể
    total_time = time.time() - start_time_total
    print(f"\nTổng thời gian chạy thí nghiệm: {total_time:.2f} giây")
    
    print("\nHoàn thành tất cả thí nghiệm!")


if __name__ == "__main__":
    # Chạy hàm chính
    comprehensive_main()


==== Thí nghiệm với bộ dữ liệu kích thước 10 dataset ====

=== Thí nghiệm với 8-puzzle (kích thước 10 dataset) ===

Chạy các thuật toán không thông tin...

Đang chạy thuật toán: BFS
Đang giải puzzle 1/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 59500, Thời gian: 0.61s
Đang giải puzzle 2/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 60045, Thời gian: 0.40s
Đang giải puzzle 3/10 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành công, Nút thăm: 109475, Thời gian: 0.83s
Đang giải puzzle 4/10 với thuật toán BFS (timeout: 60s)...
  Kết quả: Thành công, Nút thăm: 24572, Thời gian: 0.18s
Đang giải puzzle 5/10 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành công, Nút thăm: 46253, Thời gian: 0.40s
Đang giải puzzle 6/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 153257, Thời gian: 1.19s
Đang giải puzzle 7/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 63531, Thời gian: 0.51s
Đan

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Đã lưu 6 biểu đồ vào thư mục figures/

Độ khó của các 15-puzzle:
   Puzzle ID  Misplaced Tiles  Path Length Estimate  Combined Score
0          0               13                  19.5           16.25
1          1               14                  21.0           17.50
2          2               14                  21.0           17.50

Thời gian chạy cho dataset kích thước 10: 422.94 giây

==== Thí nghiệm với bộ dữ liệu kích thước 100 dataset ====

=== Thí nghiệm với 8-puzzle (kích thước 100 dataset) ===

Chạy các thuật toán không thông tin...

Đang chạy thuật toán: BFS
Đang giải puzzle 1/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 39766, Thời gian: 0.32s
Đang giải puzzle 2/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 32236, Thời gian: 0.26s
Đang giải puzzle 3/10 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 48024, Thời gian: 0.28s
Đang giải puzzle 4/10 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Đã lưu 6 biểu đồ vào thư mục figures/

Độ khó của các 15-puzzle:
   Puzzle ID  Misplaced Tiles  Path Length Estimate  Combined Score
0          0               14                  21.0           17.50
1          1               15                  22.5           18.75
2          2               15                  22.5           18.75

Thời gian chạy cho dataset kích thước 100: 436.67 giây

==== Thí nghiệm với bộ dữ liệu kích thước 1000 dataset ====

=== Thí nghiệm với 8-puzzle (kích thước 1000 dataset) ===

Chạy các thuật toán không thông tin...

Đang chạy thuật toán: BFS
Đang giải puzzle 1/5 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành công, Nút thăm: 145849, Thời gian: 0.88s
Đang giải puzzle 2/5 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 21471, Thời gian: 0.17s
Đang giải puzzle 3/5 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 58345, Thời gian: 0.34s
Đang giải puzzle 4/5 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Đã lưu 6 biểu đồ vào thư mục figures/

Độ khó của các 15-puzzle:
   Puzzle ID  Misplaced Tiles  Path Length Estimate  Combined Score
2          2               13                  19.5           16.25
0          0               14                  21.0           17.50
1          1               15                  22.5           18.75

Thời gian chạy cho dataset kích thước 1000: 430.95 giây

==== Thí nghiệm với bộ dữ liệu kích thước 10000 dataset ====

=== Thí nghiệm với 8-puzzle (kích thước 10000 dataset) ===

Chạy các thuật toán không thông tin...

Đang chạy thuật toán: BFS
Đang giải puzzle 1/5 với thuật toán BFS (timeout: 15s)...
  Kết quả: Thành công, Nút thăm: 109349, Thời gian: 0.63s
Đang giải puzzle 2/5 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành công, Nút thăm: 111891, Thời gian: 0.63s
Đang giải puzzle 3/5 với thuật toán BFS (timeout: 30s)...
  Kết quả: Thành công, Nút thăm: 9924, Thời gian: 0.05s
Đang giải puzzle 4/5 với thuật toán BFS (timeout: 60s)...
  Kết quả: Th

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Đã lưu 6 biểu đồ vào thư mục figures/

Độ khó của các 15-puzzle:
   Puzzle ID  Misplaced Tiles  Path Length Estimate  Combined Score
0          0               14                  21.0            17.5
1          1               14                  21.0            17.5
2          2               14                  21.0            17.5

Thời gian chạy cho dataset kích thước 10000: 407.85 giây
Đã lưu kết quả vào file results/performance_8puzzle_all_datasets.csv
Đã lưu phiên bản dễ đọc vào file results/performance_8puzzle_all_datasets_prettified.txt

=== Thống kê tổng hợp cho 8-puzzle ===

Top 3 thuật toán có tỷ lệ thành công cao nhất:
              Algorithm  Success Rate  Avg. Nodes Explored  \
5  Bidirectional Search           1.0               1140.6   
5  Bidirectional Search           1.0               1278.3   
5  Bidirectional Search           1.0               1640.2   

   Avg. Execution Time (s)  Avg. Solution Length  
5                 0.005500                  21.4  
5       